<font color="#506F87" size=5.5><b>Comparación de los métodos empleados</b></font>
<br>
En este notebook se va a llevar a cabo la comparación de rendimiento en términos computacionales y de resultados de los distintos métodos y distancias empleadas. Esto incluye:
1. SIFT
2. ORB
3. VGG-16 (con norma euclídea y del coseno)
4. Histograma de color (con norma euclídea y del coseno)
5. Bag of words

<font color="#506F87" size=4.5><b>Autores</b></font>
- Mario Serrano Sanchéz
- Pablo Gómez Martín
- Cristina Sandoval Nuño
- Gonzalo Rivera del Pino

<font color="#506F87" size=5.5><b>Librerías</b></font>

In [1]:
import random
from time import time
import pandas as pd
from torchvision import models

from retrieval_functions import correct_percentage, return_n_nearest_CNN, return_n_nearest_hist
from retrieval_functions import return_n_nearest_ORB, return_n_nearest_SIFT, return_n_nearest_bow

## <font color="#506F87" size=5.5><b>Cargamos el dataset con las features</b></font>

In [2]:
df_images = pd.read_pickle('Images_df.pkl', compression={'method': 'gzip', 'compresslevel': 3, 'mtime': 1})
df_features = pd.read_pickle('Features_df.pkl', compression={'method': 'gzip', 'compresslevel': 3, 'mtime': 1})

df = pd.concat([df_images, df_features], axis=1)

In [3]:
class_names = df.label.unique()

## <font color="#506F87" size=5.5><b>Comparación</b></font>

In [4]:
model = models.vgg16(weights='DEFAULT')

In [ ]:
percentages = [0]*8
times = [0]*8
functions = [return_n_nearest_SIFT, return_n_nearest_ORB, return_n_nearest_CNN, return_n_nearest_CNN, return_n_nearest_hist, 
             return_n_nearest_hist, return_n_nearest_bow, return_n_nearest_bow]

for label in class_names:

    # Para hacer la comparativa usaremos una imagen random de cada clase
    query_image = df.iloc[random.choice(df[df.label == label].index), 0]

    for i in range(8):
        if i==2 or i==3:
            if i==2:
                time1 = time()
                nearest = functions[i](query_image, df, model, norm='euclidean')
                percentage =  correct_percentage(label, nearest, df)
                time2 = time()
            else:
                time1 = time()
                nearest = functions[i](query_image, df, model, norm='cosine')
                percentage =  correct_percentage(label, nearest, df)
                time2 = time()
        
        elif i==5:
            time1 = time()
            nearest = functions[i](query_image, df, norm='cosine')
            percentage =  correct_percentage(label, nearest, df)
            time2 = time()
        
        elif i==7:
            time1 = time()
            nearest = functions[i](query_image, df, norm='cosine')
            percentage =  correct_percentage(label, nearest, df)
            time2 = time()
        
        else:
            time1 = time()
            nearest = functions[i](query_image, df)
            percentage =  correct_percentage(label, nearest, df)
            time2 = time()
                

        times[i] += time2-time1
        percentages[i] += percentage
    

In [15]:
times = [round(t/15, 2) for t in times]
percentages = [f'{round(p/15, 2)} %' for p in percentages]
methods = ['SIFT', 'ORB', 'CNN (euclidean)', 'CNN (cosine)', 'histogram (euclidean)', 'histogram (cosine)', 'BOW (euclidean)', 'BOW (cosine)']

comparison = pd.DataFrame(zip(times, percentages), index=[methods], columns=['Time', 'Succes rate']).T

comparison

,SIFT,ORB,CNN (euclidean),CNN (cosine),histogram (euclidean),histogram (cosine),BOW (euclidean),BOW (cosine)
Time,3.16,15.23,0.21,0.37,0.03,0.06,0.07,0.07
Succes rate,16.67 %,16.67 %,49.33 %,64.0 %,13.33 %,14.0 %,32.67 %,38.0 %


In [20]:
comparison.to_excel('comparison.xlsx')